# <font color=green>*BCBIO 490*</font>

In [51]:
import numpy as np
import os
import math
from Bio import SeqIO

class Sequence_Alignment:
    
    def __init__(self, match=10, mismatch=-20, gapOpen=40, gapExtend=2, diff_block=20, ambiguous=0):
        self.gapOpen = abs(gapOpen)
        self.gapExtend = abs(gapExtend)
        self.diff = abs(diff_block)
        self.sub_mat = np.zeros((129,129))
        for i in range(len(self.sub_mat)):
            for j in range(len(self.sub_mat[i])):
                if i == 'N' or j == 'N':
                    self.sub_mat[i][j] = 0 
                elif i != j:
                    self.sub_mat[i][j] = mismatch
                else:
                    self.sub_mat[i][j] = match
    
    def getScore(self, nuc1, nuc2):
        return self.sub_mat[ord(nuc1.upper())][ord(nuc2.upper())]
    
    def setGapOpenPenalty(self, penalty):
        self.gapOpen = abs(penalty)
        
    def setGapExtensionPenalty(self, penalty):
        self.gapExtend = abs(penalty)
        
    def setDifferenceBlockPenalty(self, penalty):
        self.diff = abs(penalty)
        
    def alignInput(self, fasta1, fasta2, align_type):
        """
        cwd = os.getcwd()  # Get the current working directory (cwd)
        files = os.listdir(cwd)  # Get all the files in that directory
        print("Files in %r: %s" % (cwd, files))
        """
        if os.path.getsize(fasta1) == 0 or os.path.getsize(fasta2) == 0:
            return "Input contains empty file(s)"
        if align_type != 'global' and align_type != 'local':
            return "This program only has two alignment types: 'global' or 'local'"
                    
        with open(fasta1) as handle1, open(fasta2) as handle2:
            for record1, record2 in zip(SeqIO.parse(handle1, "fasta"), SeqIO.parse(handle2, "fasta")):
                print("Aligning " + record1.id + " with " + record2.id + " using " + align_type + " alignment: ")
                print()
                if align_type == 'global':
                    self.global_alignment(record1.seq, record2.seq)
                elif align_type == 'local':
                    self.local_alignment(record1.seq, record2.seq)

                    
    def global_alignment_trace(self, seq1, seq2):
        if len(seq1) == 0 or len(seq2) == 0:
            return
        
        s_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        i_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        d_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        
        d_mat[len(seq1)][len(seq2)] = s_mat[len(seq1)][len(seq2)] - self.gapOpen
        i_mat[len(seq1)][len(seq2)] = s_mat[len(seq1)][len(seq2)] - self.gapOpen
        for i in range(len(seq2) - 1, -1, -1):
            i_mat[len(seq1)][i] = i_mat[len(seq1)][i + 1] - self.gapExtend
            s_mat[len(seq1)][i] = i_mat[len(seq1)][i]
            d_mat[len(seq1)][i] = s_mat[len(seq1)][i] - self.gapOpen
        
        for j in range(len(seq1) - 1, -1, -1):
            d_mat[j][len(seq2)] = d_mat[j + 1][len(seq2)] - self.gapExtend
            s_mat[j][len(seq2)] = d_mat[j][len(seq2)]
            i_mat[j][len(seq2)] = s_mat[j][len(seq2)] - self.gapOpen
            
            for k in range(len(seq2) - 1, -1, -1):
                d_mat[j][k] = max(d_mat[j + 1][k] - self.gapExtend, s_mat[j + 1][k] - self.gapOpen - self.gapExtend)
                i_mat[j][k] = max(i_mat[j][k + 1] - self.gapExtend, s_mat[j][k + 1] - self.gapOpen - self.gapExtend)
                s_mat[j][k] = max(s_mat[j + 1][k + 1] + self.getScore(seq1[j], seq2[k]), d_mat[j][k], i_mat[j][k])
                
        current = 'S'
        i = j = 0
        align_seq1 = ""
        align_seq2 = ""
        align_mid = ""

        while i <= len(seq1) and j <= len(seq2):
            if current == 'S':
                if i == len(seq1) and j == len(seq2):
                    break
                elif i == len(seq1) or s_mat[i][j] == i_mat[i][j]:
                    current = 'I'
                    continue
                elif j == len(seq2) or s_mat[i][j] == d_mat[i][j]:
                    current = 'D'
                    continue
                align_seq1 += seq1[i]
                align_seq2 += seq2[j]
                if seq1[i].upper() != seq2[j].upper():
                    align_mid += "*"
                else:
                    align_mid += "|"
                i += 1
                j += 1
                continue
                
            elif current == 'I':
                align_seq1 += '-'
                align_seq2 += seq2[j]
                align_mid += " "
                if (j == len(seq2) - 1) or (i_mat[i][j] == s_mat[i][j + 1] - self.gapOpen - self.gapExtend):
                    current = 'S'
                j += 1
                continue
                
            elif current == 'D':
                align_seq1 += seq1[i]
                align_seq2 += '-'
                align_mid += " "
                if (i == len(seq1) - 1) or (d_mat[i][j] == s_mat[i + 1][j] - self.gapOpen - self.gapExtend):
                    current = 'S'
                i += 1
                continue
        
        return [s_mat[0][0], align_seq1, align_mid, align_seq2]
    
    def global_alignment(self, seq1, seq2):
        arr = self.global_alignment_trace(seq1, seq2)
        self.printAlignment(seq1, seq2, arr[0], arr[1], arr[2], arr[3])
        
    def printAlignment(self, seq1, seq2, score, align_seq1, align_mid, align_seq2):
        counter = 0
        curr1 = 1
        curr2 = 1
        max_space = len(str(min(len(seq1), len(seq2))))
        print("Alignment Score: " + str(score))
        print("Similarity: " + str(round((align_mid.count("|")/ len(align_mid)) * 100,2)) + "%\n")
        while counter <= len(align_seq1):
            print("Sequence 1 > " + self.generateString('left', max_space, str(curr1)) + align_seq1[counter: counter + 79] + self.generateString('right', max_space, str(min(curr1 + 79 - align_seq1[counter: counter + 79].count('-') - 1, len(seq1)))))
            print("             " + self.generateString('left', max_space, "") + align_mid[counter: counter + 79] + self.generateString('right', max_space, ""))
            print("Sequence 2 > " + self.generateString('left', max_space, str(curr2)) + align_seq2[counter: counter + 79] + self.generateString('right', max_space, str(min(curr2 + 79 - align_seq2[counter: counter + 79].count('-') - 1, len(seq2)))))
            
            print("\n")
            curr1 = min(curr1 + 79 - align_seq1[counter: counter + 79].count('-'), len(seq1))
            curr2 = min(curr2 + 79 - align_seq2[counter: counter + 79].count('-'), len(seq2))
            counter += 79
    
    def local_alignment(self, seq1, seq2):
        if len(seq1) == 0 or len(seq2) == 0:
            return
        
        highestScore = 0
        firstRow = len(seq1)
        firstCol = len(seq2)
        s_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        i_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        d_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        
        d_mat[len(seq1)][len(seq2)] = -(self.gapOpen + self.gapExtend)
        i_mat[len(seq1)][len(seq2)] = -(self.gapOpen + self.gapExtend)
        for i in range(len(seq2) - 1, -1, -1):
            i_mat[len(seq1)][i] =  -(self.gapOpen + self.gapExtend)
            d_mat[len(seq1)][i] =  -(self.gapOpen + self.gapExtend)
        
        for j in range(len(seq1) - 1, -1, -1):
            d_mat[j][len(seq2)] =  -(self.gapOpen + self.gapExtend)
            i_mat[j][len(seq2)] =  -(self.gapOpen + self.gapExtend)
        
            for k in range(len(seq2) - 1, -1, -1):
                d_mat[j][k] = max(d_mat[j + 1][k] - self.gapExtend, s_mat[j + 1][k] - self.gapOpen - self.gapExtend)
                i_mat[j][k] = max(i_mat[j][k + 1] - self.gapExtend, s_mat[j][k + 1] - self.gapOpen - self.gapExtend)
                s_mat[j][k] = max(0, s_mat[j + 1][k + 1] + self.getScore(seq1[j], seq2[k]), d_mat[j][k], i_mat[j][k])
                
                if highestScore < s_mat[j][k]:
                    highestScore = s_mat[j][k]
                    firstRow = j
                    firstCol = k
                    
        current = 'S'
        i = firstRow
        j = firstCol 
        align_seq1 = ""
        align_seq2 = ""
        align_mid = ""

        while i <= len(seq1) and j <= len(seq2):
            if current == 'S':
                if i == len(seq1) or j == len(seq2) or s_mat[i][j] == 0:
                    break
                elif s_mat[i][j] == i_mat[i][j]:
                    current = 'I'
                    continue
                elif s_mat[i][j] == d_mat[i][j]:
                    current = 'D'
                    continue
                align_seq1 += seq1[i]
                align_seq2 += seq2[j]
                if seq1[i].upper() != seq2[j].upper():
                    align_mid += "*"
                else:
                    align_mid += "|"
                i += 1
                j += 1
                continue
                
            elif current == 'I':
                align_seq1 += '-'
                align_seq2 += seq2[j]
                align_mid += " "
                if (j == len(seq2) - 1) or (i_mat[i][j] == s_mat[i][j + 1] - self.gapOpen - self.gapExtend):
                    current = 'S'
                j += 1
                continue
                
            elif current == 'D':
                align_seq1 += seq1[i]
                align_seq2 += '-'
                align_mid += " "
                if (i == len(seq1) - 1) or (d_mat[i][j] == s_mat[i + 1][j] - self.gapOpen - self.gapExtend):
                    current = 'S'
                i += 1
                continue
                
        lastRow = i
        lastCol = j
        counter = 0
        curr1 = firstRow + 1
        curr2 = firstCol + 1
        max_space = len(str(max(len(seq1), len(seq2))))
        print("Alignment Score: " + str(s_mat[firstRow][firstCol]) + "\n")
        while counter <= len(align_seq1):
            print("Sequence 1 > " + self.generateString('left', max_space, str(curr1)) + align_seq1[counter: counter + 79] + self.generateString('right', max_space, str(min(curr1 + 79 - align_seq1[counter: counter + 79].count('-') - 1, lastRow))))
            print("             " + self.generateString('left', max_space, "") + align_mid[counter: counter + 79] + self.generateString('right', max_space, ""))
            print("Sequence 2 > " + self.generateString('left', max_space, str(curr2)) + align_seq2[counter: counter + 79] + self.generateString('right', max_space, str(min(curr2 + 79 - align_seq2[counter: counter + 79].count('-') - 1, lastCol))))
            print("\n")
            curr1 = min(curr1 + 79 - align_seq1[counter: counter + 79].count('-'), lastRow)
            curr2 = min(curr2 + 79 - align_seq2[counter: counter + 79].count('-'), lastCol)
            counter += 79
            
    
    # Do we need value of H_MAT, h_mat, D_MAT, and d_mat? They are not 0 at start. 
    def gap3_trace(self, seq1, seq2, s, d, h, S, D, H):

        if len(seq1) <= 50:
            arr = self.global_alignment_trace(seq1, seq2)
            return arr
        
        imid = len(seq1) // 2
        
        s_mat = np.zeros((imid + 1, len(seq2) + 1))
        i_mat = np.zeros((imid + 1, len(seq2) + 1))
        d_mat = np.zeros((imid + 1, len(seq2) + 1))
        h_mat = np.zeros((imid + 1, len(seq2) + 1))
        
        S_MAT = np.zeros((len(seq1) - imid + 1, len(seq2) + 1))
        I_MAT = np.zeros((len(seq1) - imid + 1, len(seq2) + 1))
        D_MAT = np.zeros((len(seq1) - imid + 1, len(seq2) + 1))
        H_MAT = np.zeros((len(seq1) - imid + 1, len(seq2) + 1))
        
        s_mat[0][0] = s
        i_mat[0][0] = s_mat[0][0] - self.gapOpen
        d_mat[0][0] = d
        h_mat[0][0] = h
        
        S_MAT[len(seq1) - imid][len(seq2)] = S
        I_MAT[len(seq1) - imid][len(seq2)] = S_MAT[len(seq1) - imid][len(seq2)] - self.gapOpen
        H_MAT[len(seq1) - imid][len(seq2)] = H
        D_MAT[len(seq1) - imid][len(seq2)] = D
        
        for j in range(1, len(seq2) + 1):
            i_mat[0][j] = i_mat[0][j-1] - self.gapExtend
            h_mat[0][j] = -self.diff
            s_mat[0][j] = max(i_mat[0][j], h_mat[0][j])
            d_mat[0][j] = s_mat[0][j] - self.gapOpen
            
        for i in range(1, imid + 1):
            d_mat[i][0] = d_mat[i-1][0] - self.gapExtend
            h_mat[i][0] = -self.diff
            s_mat[i][0] = max(d_mat[i][0], h_mat[i][0])
            i_mat[i][0] = s_mat[i][0] - self.gapOpen
            
            for j in range(1, len(seq2) + 1):
                d_mat[i][j] = max(d_mat[i - 1][j] - self.gapExtend, s_mat[i - 1][j] - self.gapOpen - self.gapExtend)
                i_mat[i][j] = max(i_mat[i][j - 1] - self.gapExtend, s_mat[i][j - 1] - self.gapOpen - self.gapExtend)
                h_mat[i][j] = max(h_mat[i][j - 1], h_mat[i - 1][j], s_mat[i][j - 1] - self.diff, s_mat[i - 1][j] - self.diff)
                s_mat[i][j] = max(s_mat[i - 1][j - 1] + self.getScore(seq1[i-1], seq2[j-1]), d_mat[i][j], i_mat[i][j], h_mat[i][j])
            
        for j in range(len(seq2) - 1, -1, -1):
            I_MAT[len(seq1) - imid][j] = I_MAT[len(seq1) - imid][j + 1] - self.gapExtend
            H_MAT[len(seq1) - imid][j] = -self.diff
            S_MAT[len(seq1) - imid][j] = max(I_MAT[len(seq1) - imid][j], H_MAT[len(seq1) - imid][j])
            D_MAT[len(seq1) - imid][j] = S_MAT[len(seq1) - imid][j] - self.gapOpen 
            
        for i in range(len(seq1) - imid - 1, -1, -1):
            D_MAT[i][len(seq2)] = D_MAT[i + 1][len(seq2)] - self.gapExtend
            H_MAT[i][len(seq2)] = -self.diff
            S_MAT[i][len(seq2)] = max(D_MAT[i][len(seq2)], H_MAT[i][len(seq2)])
            I_MAT[i][len(seq2)] = S_MAT[i][len(seq2)] - self.gapOpen
            
            for j in range(len(seq2) - 1, -1, -1):
                D_MAT[i][j] = max(D_MAT[i + 1][j] - self.gapExtend, S_MAT[i + 1][j] - self.gapOpen - self.gapExtend)
                I_MAT[i][j] = max(I_MAT[i][j + 1] - self.gapExtend, S_MAT[i][j + 1] - self.gapOpen - self.gapExtend)
                H_MAT[i][j] = max(H_MAT[i][j + 1], H_MAT[i + 1][j], S_MAT[i][j + 1] - self.diff, S_MAT[i + 1][j] - self.diff)
                S_MAT[i][j] = max(S_MAT[i + 1][j + 1] + self.getScore(seq1[i + imid - 1], seq2[j-1]), D_MAT[i][j], I_MAT[i][j], H_MAT[i][j])
        
        hk = -math.inf
        df = -math.inf
        st = -math.inf
        
        jh = 0
        jd = 0
        js = 0
        
        for j in range(len(seq2) + 1):
            if h_mat[imid][j] + H_MAT[0][j] + self.diff > hk:
                hk = h_mat[imid][j] + H_MAT[0][j] + self.diff
                jh = j
            if d_mat[imid][j] + D_MAT[0][j] + self.gapOpen > df:
                df = d_mat[imid][j] + D_MAT[0][j] + self.gapOpen
                jd = j
            if s_mat[imid][j] + S_MAT[0][j] > st:
                st = s_mat[imid][j] + S_MAT[0][j]
                js = j
        
        # What if there are two with the same value? For example, hk = -10 and st = -10, do we prefer one over the 
        # other? 
        if max(hk, df, st) == hk:
            jmid = jh
            
        elif max(hk, df, st) == df:
            jmid = jd
            
        else:
            jmid = js
            
        
        top_align = self.gap3_trace(seq1[0: imid + 1], seq2[0:jmid + 1], s_mat[0][0], d_mat[0][0], h_mat[0][0], 
                                     S_MAT[0][jmid], D_MAT[0][jmid], H_MAT[0][jmid])
        
        bottom_align = self.gap3_trace(seq1[imid+1::], seq2[jmid+1::], s_mat[len(s_mat) - 1][jmid], 
                                        d_mat[len(d_mat) - 1][jmid], h_mat[len(h_mat) - 1][jmid], 
                                        S_MAT[len(S_MAT) - 1][len(S_MAT[0]) - 1],
                                        D_MAT[len(D_MAT) - 1][len(D_MAT[0]) - 1],
                                        H_MAT[len(H_MAT) - 1][len(H_MAT[0]) - 1])
        
        final_score = top_align[0] + bottom_align[0]
        final_align_seq1 = top_align[1] + bottom_align[1]
        final_align_mid = top_align[2] + bottom_align[2]
        final_align_seq2 = top_align[3] + bottom_align[3]
        
        return [final_score, final_align_seq1, final_align_mid, final_align_seq2]
        
        
    def gap3(self, seq1, seq2):
        s = 0
        d = -self.gapOpen
        h = -self.diff
        arr = self.gap3_trace(seq1, seq2, s, d, h, s, d, h)
        self.printAlignment(seq1, seq2, arr[0], arr[1], arr[2], arr[3])
        
        
    def generateString(self, position, max_space, string):
        if len(string) > max_space:
            return
        if position == 'left':
            return string + ((max_space - len(string)) * " ") + " "
        elif position == 'right':
            return " " + ((max_space - len(string)) * " ") + string
        else:
            return


In [54]:
p = Sequence_Alignment()
A = "ATCGACATCGTAGCTGATCGATGTACGATCGATCGATCGATCGATCGATCGATCGCTAGCTGATGCTAGCTAGCTTAAAAT"
B = "TAGACACGTAGCTGATCGATGTACGATCGATCGAGATCGATCGATCGATCGCTAACTGATGCTAGCTAGCTTAT"
print(len(A))
print(len(B))
p.gap3(A, B)
p.global_alignment(A, B)

81
74
Alignment Score: 506.0
Similarity: 88.89%

Sequence 1 > 1  ATCGACATCGTAGCTGATCGATGTACGATCGATCGATCGATCGATCGATCGATCGCTAGCTGATGCTAGCTAGCTTAAA 79
                 |*|||| ||||||||||||||||||||||||||||  ||||||||||||||||||||*|||||||||||||||||      
Sequence 2 > 1  -TAGACA-CGTAGCTGATCGATGTACGATCGATCGA--GATCGATCGATCGATCGCTAACTGATGCTAGCTAGCTT--- 72


Sequence 1 > 80 AT 81
                ||   
Sequence 2 > 73 AT 74


Alignment Score: 506.0
Similarity: 88.89%

Sequence 1 > 1  ATCGACATCGTAGCTGATCGATGTACGATCGATCGATCGATCGATCGATCGATCGCTAGCTGATGCTAGCTAGCTTAAA 79
                 |*|||| ||||||||||||||||||||||||||||  ||||||||||||||||||||*|||||||||||||||||      
Sequence 2 > 1  -TAGACA-CGTAGCTGATCGATGTACGATCGATCGA--GATCGATCGATCGATCGCTAACTGATGCTAGCTAGCTT--- 72


Sequence 1 > 80 AT 81
                ||   
Sequence 2 > 73 AT 74


